In [66]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import statsmodels.api as sm
from sklearn.cluster import KMeans
from factor_analyzer import FactorAnalyzer


In [81]:
start = '2020-01-02'
SP500_list = list(pd.read_csv("../constituents.csv")['Symbol'])
df_close = yf.download(SP500_list[:500], start=start, end=pd.to_datetime(start)+timedelta(days=1), progress=False)
df_close_t = df_close['Volume'].T
df_close_t.columns = ['Volume']
df_close_t.dropna(inplace=True)
list_ = df_close_t.sort_values(by='Volume', ascending=False)[:100].index.to_list()
list_[:5], len(list_)


9 Failed downloads:
['CARR', 'SOLV', 'GEHC', 'ABNB', 'CEG', 'OTIS', 'KVUE', 'VLTO', 'GEV']: Exception("%ticker%: Data doesn't exist for startDate = 1577941200, endDate = 1578027600")


(['TSLA', 'AAPL', 'AMZN', 'AMD', 'F'], 100)

In [86]:
stock_list = list_

df_close = yf.download(stock_list, start=pd.to_datetime(start) - timedelta(days=365*5), end=start, progress=False)['Adj Close']
spx_close = yf.download("^GSPC", start=pd.to_datetime(start) - timedelta(days=365*5), end=start, progress=False)['Adj Close']
df_close.head()

Ticker,AAL,AAPL,ABBV,AMAT,AMCR,AMD,AMZN,BAC,BBWI,BMY,...,UBER,V,VTRS,VZ,WBA,WDC,WFC,WMB,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,51.051491,23.714724,43.480469,21.407751,7.637547,2.66,15.1095,14.368293,48.551880,44.333237,...,NaN,60.649261,49.342518,29.074713,54.317139,91.529427,40.443760,24.864410,23.491823,59.353569
2015-01-06,50.255589,23.716957,43.265255,20.601412,7.637547,2.63,14.7645,13.938403,48.053535,44.257565,...,NaN,60.258427,47.978783,29.368139,54.455673,89.488350,39.599911,24.526627,23.672848,59.038029
2015-01-07,50.227161,24.049520,45.013893,20.796350,7.310223,2.58,14.9210,14.004541,49.863632,44.832718,...,NaN,61.065800,49.687881,29.178633,55.848225,90.832253,39.835583,24.394947,24.300941,59.636238
2015-01-08,50.843033,24.973553,45.484676,21.460915,7.310223,2.61,15.0230,14.293894,49.651688,45.952785,...,NaN,61.884827,50.166077,29.804024,56.540878,92.713753,40.717442,24.629673,24.813843,60.628876
2015-01-09,49.289124,25.000336,44.240456,21.478634,7.310223,2.63,14.8465,14.037606,48.907040,45.650070,...,NaN,60.967506,49.448784,29.538702,55.170158,94.259254,40.048447,24.320524,24.506651,60.543381


In [88]:
df_close_pct = df_close.pct_change().iloc[1:, :]
spx_pct = spx_close.pct_change()

df_close_pct.dropna(inplace=True, axis=1)
spx_pct.dropna(inplace=True)

df_close_pct_m = df_close_pct.resample("ME").agg(lambda x: (x+1).prod() - 1)
spx_pct_m = spx_pct.resample("ME").agg(lambda x: (x+1).prod() - 1)

In [ ]:
factor = 9
res_df = pd.DataFrame()
alpha_dict = {}
X = spx_pct_m
X_sm = sm.add_constant(X)
for ticker in df_close_pct_m.columns:
    # OLS regression
    close = df_close_pct_m[ticker]
    y = close
    model = sm.OLS(y, X_sm)

    res = model.fit()
    residuals = res.resid
    res_df[ticker] = residuals

    # store alpha
    res_table = res.summary().tables
    alpha = float(res_table[1].data[1][1].strip())
    alpha_dict[ticker] = alpha
factor_analyzer = FactorAnalyzer(n_factors=factor, rotation='promax') # varimax
factor_analyzer.fit(res_df.corr())

# Get factor loadings
factor_loadings = factor_analyzer.loadings_
df = pd.DataFrame(factor_loadings, index=list(res_df.columns))
df['Ticker'] = df.index
# Apply K-means clustering
kmeans = KMeans(n_clusters=factor)
kmeans.fit(df.iloc[:, :factor])

# Assign cluster labels to tickers
df['Cluster'] = kmeans.labels_

# Display the result
res = df[['Ticker', 'Cluster']]
res = res.copy()
# add clustered value to check further
cluster_value = []
for i in range(len(res)):
    classification = res.iloc[i, 1]
    cluster_value.append(df.iloc[i, classification])
res['Cluster_value'] = cluster_value

sorted_res = res.sort_values(by='Cluster')


res_dict = {}
for i in range(factor):

    res_dict["class" + str(i)] = list(sorted_res[sorted_res['Cluster'] == i]['Ticker'])
res_dict
# factor == 3比較穩定 但還是不合理

In [93]:
import json
with open("farrell_at2020_top100_cluster9.json", "w") as f:
    json.dump(res_dict, f, indent=4)

In [94]:
# classification index
import numpy as np
indexes = pd.DataFrame()
for key, value in res_dict.items():
    print(value)
    index_close = df_close[value]
    index_close = index_close/len(value)
    indexes[f"{str(key)}-index"] = index_close.sum(axis=1)

indexes_pct = indexes.pct_change()
indexes_pct.dropna(inplace=True)

indexes_pct_m = indexes_pct.resample("ME").agg(lambda x: (x+1).prod() - 1)
mean = np.mean(indexes_pct_m, axis=0)
sd = np.std(indexes_pct_m, axis=0)
reg_coef = []
for i in range(len(indexes_pct_m.columns)):
    X = spx_pct_m
    X_sm = sm.add_constant(X)
    y = indexes_pct_m.iloc[:, i]
    model = sm.OLS(y, X_sm)
    res = model.fit()
    reg_coef.append(res.summary().tables[1].data[2][1].replace(" ", ""))


result = pd.DataFrame()
result.index = list(indexes.columns)
result["mean"] = mean
result["sd"] = sd
result["reg_coef"] = reg_coef
result.sort_values(by='reg_coef', ascending=False)


['AAL', 'GM', 'INTC', 'LVS', 'MCHP', 'NVDA', 'MU', 'AMAT', 'VTRS', 'WDC', 'ON']
['NEM', 'EQT', 'T', 'KMI', 'HST', 'FCX', 'CVX', 'XOM', 'CTRA']
['SBUX', 'PM', 'PG', 'MSFT', 'MDLZ', 'KO', 'WMT', 'JNJ', 'CMCSA', 'GOOGL', 'GOOG', 'GLW', 'MO', 'GEN']
['BAC', 'WFC', 'TFC', 'SCHW', 'RF', 'BBWI', 'MS', 'C', 'KEY', 'IVZ', 'JPM', 'HBAN']
['AMZN', 'AMD', 'MRK', 'CVS', 'PFE', 'BSX', 'GILD', 'TSLA', 'ABBV', 'WBA', 'BMY']
['HAL', 'OXY', 'GE', 'WMB', 'SLB', 'PARA', 'DVN', 'MRO']
['CSCO', 'AAPL', 'CPRT', 'V', 'CSX', 'CRM', 'AMCR', 'DD', 'QCOM', 'FTNT', 'DIS', 'HPQ', 'ORCL', 'META', 'PCG', 'F', 'ENPH']
['KR', 'TGT', 'TJX', 'FI', 'NKE']
['PPL', 'NEE', 'DOC', 'VZ', 'KIM', 'EXC', 'IRM']


,mean,sd,reg_coef
class0-index,0.007487,0.069400,1.6199
class5-index,-0.005647,0.067194,1.3932
class3-index,0.008247,0.058708,1.2899
class6-index,0.012042,0.043125,1.0880
class4-index,0.008400,0.044674,0.9916
class1-index,0.001207,0.048005,0.8614
class2-index,0.011310,0.033611,0.7768
class7-index,0.013874,0.041121,0.7037
class8-index,0.007356,0.035044,0.3753


In [107]:
# check for correlation after auto factorized
import numpy as np
cov = res_df.corr()
for cls in res_dict.values():
    res_df_temp = res_df[cls]
    corr = res_df_temp.corr()
    list_corr = []
    for i in range(len(corr)):
        for j in range(i):
            list_corr.append(corr.iloc[i, j])
    print(np.mean(list_corr))
# delet anything with correlation < 0.15

0.2146777870261069
0.5969672957724629
0.1391714593330671


In [12]:
cov = res_df.corr()
# correlation matrix cleaning
for i in range(len(cov)):
    for j in range(i+1):
        cov.iloc[i, j] = 0
classi = 3
for i in range(3):
    max_value = cov.max().max()  # Get the maximum value across all rows and columns
    max_row, max_col = cov.stack().idxmax()  
    print(max_value, max_row, max_col)

    cov.drop([max_row, max_col], inplace=True)
    cov.drop(columns=[max_row, max_col], inplace=True)

0.799800153916491 AMAT LRCX
0.536247517061427 DXCM PODD
0.304298809043346 DECK ETSY
